In [1]:
from helper import *

pd.set_option('display.max_rows', None)

In [2]:
# load preds
preds_2025 = pd.read_csv('./data/clean/predictions_2025.csv')

# load fantasy data
fantasy_2024 = pd.read_csv('./data/clean/pff_fantasy_data.csv').query('Year == 2024')[['Player', 'PPGPosRank_half-ppr', 'grades_offense']]

# add 'PPGPosRank_half-ppr' as 'rank_2024' col in preds_2025
preds_2025 = preds_2025.merge(fantasy_2024, how='left', left_on='player', right_on='Player').drop(columns=['Player'])
preds_2025 = preds_2025.rename(columns={'PPGPosRank_half-ppr': 'rank_2024', 'grades_offense': 'grade_2024'})

In [10]:
pos = 'QB'
num_players = 36
preds_2025.query('pos == @pos').sort_values('pos_rank', ascending=True).head(50)

,player,team,pos,games_played_pct_pred,ppg_pred,adj_pred,pos_rank,rank_2024,grade_2024
0,Lamar Jackson,BAL,QB,0.738276,20.971642,15.482856,1.0,1,94.0
1,Josh Allen,BUF,QB,0.826039,18.648424,15.404331,2.0,2,91.8
2,Patrick Mahomes,KAN,QB,0.822515,18.130830,14.912885,3.0,13,85.8
3,Jared Goff,DET,QB,0.828161,16.489944,13.656325,4.0,8,81.8
4,Bo Nix,DEN,QB,0.755409,17.433376,13.169337,5.0,9,76.4
5,Jayden Daniels,WAS,QB,0.722119,17.145458,12.381065,6.0,6,89.6
6,Joe Burrow,CIN,QB,0.744637,16.511486,12.295066,7.0,3,93.9
9,Jalen Hurts,PHI,QB,0.695946,17.548178,12.212588,8.0,5,76.3
11,Baker Mayfield,TAM,QB,0.732111,16.157238,11.828886,9.0,4,86.0
13,Kyler Murray,ARI,QB,0.640275,17.686892,11.324471,10.0,14,82.0


In [13]:
teams_2024 = pd.read_csv('./data/pff/team/2024.csv')
teams_2024['blocking grade'] = (teams_2024['Pass Blocking Grade'] + teams_2024['Run Blocking Grade']).div(2)
teams_2024 = teams_2024[['Tm', 'Points For', 'Points Against', 'blocking grade', 'Pass Blocking Grade', 'Run Blocking Grade']].copy().sort_values('blocking grade', ascending=False).reset_index(drop=True)
teams_2024

,Tm,Points For,Points Against,blocking grade,Pass Blocking Grade,Run Blocking Grade
0,Atlanta Falcons,389,423,77.80,72.8,82.8
1,San Francisco 49ers,389,436,76.00,72.0,80.0
2,Denver Broncos,425,311,75.65,83.6,67.7
3,Indianapolis Colts,377,427,74.75,72.5,77.0
4,Philadelphia Eagles,463,303,74.55,73.9,75.2
5,Tampa Bay Buccaneers,502,385,74.10,82.5,65.7
6,Detroit Lions,564,342,73.40,68.4,78.4
7,Kansas City Chiefs,385,326,72.15,70.7,73.6
8,Green Bay Packers,460,338,72.10,82.1,62.1
9,Arizona Cardinals,400,379,71.20,75.8,66.6


In [15]:
teams_2024.sort_values('Points For', ascending=False)

,Tm,Points For,Points Against,blocking grade,Pass Blocking Grade,Run Blocking Grade
6,Detroit Lions,564,342,73.40,68.4,78.4
11,Buffalo Bills,525,368,69.60,74.7,64.5
13,Baltimore Ravens,518,361,68.40,73.3,63.5
5,Tampa Bay Buccaneers,502,385,74.10,82.5,65.7
23,Washington Commanders,485,391,61.15,64.5,57.8
29,Cincinnati Bengals,472,434,53.50,56.3,50.7
4,Philadelphia Eagles,463,303,74.55,73.9,75.2
8,Green Bay Packers,460,338,72.10,82.1,62.1
16,Minnesota Vikings,432,332,67.60,69.3,65.9
2,Denver Broncos,425,311,75.65,83.6,67.7
